In [ ]:
!pip install transformers datasets evaluate --quiet

#Download dataset from GitHub
!wget https://raw.githubusercontent.com/SamirMoustafa/nmt-with-attention-for-ar-to-en/master/ara_.txt -O ara_eng.txt


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform

In [ ]:
!pip install sacrebleu
!pip install sacremoses

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 4.8 MB/s eta 0:00:00


In [ ]:
with open("ara_eng.txt", "r", encoding="utf-8") as f:
    for i in range(10):
        print(f.readline())


Hi.	مرحبًا.

Run!	اركض!

Help!	النجدة!

Jump!	اقفز!

Stop!	قف!

Go on.	داوم.

Go on.	استمر.

Hello!	مرحباً.

Hurry!	تعجّل!

Hurry!	استعجل!



In [ ]:
# Needed libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset
import re
import evaluate
from transformers import (
    MarianMTModel,
    MarianTokenizer,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
)

# Clean text
def clean_text(text):
    if not isinstance(text, str):
        return ""

    diacritics_pattern = re.compile(r'[\u064B-\u065F]')
    text = re.sub(diacritics_pattern, '', text)
    text = text.replace("ى", "ي").replace("ة", "ه").replace("ؤ", "و")
    text = text.replace("إ", "ا").replace("أ", "ا").replace("آ", "ا").replace("ئ", "ي")
    text = text.replace("ـ", "")
    text = re.sub(r'[^\u0600-\u06FF\s؟!.،]', '', text)  # Keep Arabic letters + punctuation
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Load dataset
data = pd.read_csv("ara_eng.txt", sep='\t', header=None, names=["English", "Arabic"])
data.dropna(inplace=True)
data["Arabic"] = data["Arabic"].apply(clean_text)
data = data[data["Arabic"].str.strip() != ""]

data["English"] = data["English"].str.strip()

# Split data
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

train_data = train_data.reset_index(drop=True)
test_data = test_data.reset_index(drop=True)

# To Hugging Face Datasets
train_dataset = Dataset.from_pandas(train_data.reset_index(drop=True))
test_dataset = Dataset.from_pandas(test_data.reset_index(drop=True))

# Load model and tokenizer
model_name = "Helsinki-NLP/opus-mt-ar-en"
tokenizer = MarianTokenizer.from_pretrained(model_name)
tokenizer.padding_side = "right"
model = MarianMTModel.from_pretrained(model_name)

# Tokenization
def tokenize_function(examples):
    model_inputs = tokenizer(
        examples["Arabic"],
        max_length=256,        # adjusted (128) remember _all
        padding="max_length",
        truncation=True
    )
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["English"],
            max_length=256,      # adjusted (128) remember _all
            padding="max_length",
            truncation=True
        )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Tokenize
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/917k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.13M [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

Map:   0%|          | 0/8593 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/2149 [00:00<?, ? examples/s]

In [ ]:
# Training args
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=6,
    per_device_eval_batch_size=6,
    num_train_epochs=10,
    logging_dir="./logs",
    logging_steps=50,
    save_strategy="no",
    predict_with_generate=True,
    report_to="none",
)

# Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
)

trainer.train()

<ipython-input-7-e9ff369ac3c5>:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Step,Training Loss
50,0.010900
100,0.006900
150,0.011900
200,0.025700
250,0.021700
300,0.021500
350,0.022800
400,0.021100
450,0.021100
500,0.022400


TrainOutput(global_step=14330, training_loss=0.005076905386359934, metrics={'train_runtime': 3947.2366, 'train_samples_per_second': 21.77, 'train_steps_per_second': 3.63, 'total_flos': 5825772996526080.0, 'train_loss': 0.005076905386359934, 'epoch': 10.0})

In [ ]:
# Evaluation (sample)
chrf_metric = evaluate.load("chrf")
bleu_metric= evaluate.load("bleu")

def evaluate_model(n_samples=5):
    references = []
    predictions = []

    count = 0
    for i in range(len(test_data)):
        if count >= n_samples:
            break

        source_text = test_data.iloc[i]["Arabic"]
        target_text = test_data.iloc[i]["English"]

        if not isinstance(source_text, str) or not source_text.strip():
            continue  # Skip blank or non-string inputs

        inputs = tokenizer(
            source_text,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=256,
        ).to(model.device)

        output_ids = model.generate(
            **inputs,
            max_length=128,
            num_beams=10,
            length_penalty=1.5,
            early_stopping=True
        )
        prediction = tokenizer.decode(output_ids[0], skip_special_tokens=True)

        print(f"Arabic:    {source_text}")
        print(f"Predicted: {prediction}")
        print(f"True:      {target_text}")
        print("-" * 80)

        references.append([target_text])
        predictions.append(prediction)
        count += 1

    if len(references) > 0 and len(predictions) > 0:
        chrf_score = chrf_metric.compute(predictions=predictions, references=references)
        bleu_score = bleu_metric.compute(predictions=predictions, references=references)
        print(f"ChrF Score: {chrf_score['score']}")
        print(f"bleu score: {bleu_score['bleu']}")
    else:
        print("No valid predictions to evaluate.")


evaluate_model()

Arabic:    كان الجو حارا جدا عصر اليوم.
Predicted: It was very hot this afternoon.
True:      It was very hot this afternoon.
--------------------------------------------------------------------------------
Arabic:    هل ستكون في المنزل لتناول العشاء؟
Predicted: Will you be home for dinner?
True:      Will you be home for dinner?
--------------------------------------------------------------------------------
Arabic:    لا انصح بذلك.
Predicted: I don't recommend it.
True:      I don't recommend that.
--------------------------------------------------------------------------------
Arabic:    امي اكبر من ابي سنا.
Predicted: My mother is older than my father.
True:      Mom is older than Dad.
--------------------------------------------------------------------------------
Arabic:    اعجبت عمتي بنجاحي.
Predicted: My aunt liked my success.
True:      My aunt was pleased with my success.
--------------------------------------------------------------------------------
ChrF Score: 74.414882174

In [ ]:
# Evaluation (all)
chrf_metric = evaluate.load("chrf")
bleu_metric = evaluate.load("bleu")

def evaluate_model():
    references = []
    predictions = []

    for i in range(len(test_data)):
        source_text = test_data.iloc[i]["Arabic"]
        target_text = test_data.iloc[i]["English"]

        if not isinstance(source_text, str) or not source_text.strip():
            continue  # Skip blank or non-string inputs

        inputs = tokenizer(
            source_text,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=128,
        ).to(model.device)

        output_ids = model.generate(
            **inputs,
            max_length=256,
            num_beams=10,
            length_penalty=1.5,
            early_stopping=True
        )
        prediction = tokenizer.decode(output_ids[0], skip_special_tokens=True)

        print(f"Arabic:    {source_text}")
        print(f"Predicted: {prediction}")
        print(f"True:      {target_text}")
        print("-" * 80)

        references.append([target_text])
        predictions.append(prediction)

    if len(references) > 0 and len(predictions) > 0:
        chrf_score = chrf_metric.compute(predictions=predictions, references=references)
        bleu_score = bleu_metric.compute(predictions=predictions, references=references)
        print(f"ChrF Score: {chrf_score['score']}")
        print(f"BLEU Score: {bleu_score['bleu']}")
    else:
        print("No valid predictions to evaluate.")


evaluate_model()

Streaming output truncated to the last 5000 lines.
Arabic:    عليك ان تدرس بجد.
Predicted: You should study hard.
True:      You must study hard.
--------------------------------------------------------------------------------
Arabic:    جف القلم.
Predicted: The pen ran dry.
True:      This pen has run dry.
--------------------------------------------------------------------------------
Arabic:    ساتصل عليك بعد ظهر الغد.
Predicted: I'll call you tomorrow afternoon.
True:      I will call you tomorrow afternoon.
--------------------------------------------------------------------------------
Arabic:    توم له ذاكره ضعيفه.
Predicted: Tom has a bad memory.
True:      Tom has a poor memory.
--------------------------------------------------------------------------------
Arabic:    انك في خطر!
Predicted: You're in danger!
True:      It's dangerous!
--------------------------------------------------------------------------------
Arabic:    البس قبعتك.
Predicted: Put on your cap.
True:      

In [ ]:
# Save the model and tokenizer
model.save_pretrained("./results")
tokenizer.save_pretrained("./results")

/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[62833]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


('./results/tokenizer_config.json',
 './results/special_tokens_map.json',
 './results/vocab.json',
 './results/source.spm',
 './results/target.spm',
 './results/added_tokens.json')

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import re

# Check GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load custom trained model and tokenizer
model_name = "./results"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)


def split_text_by_sentences(text, max_chars=600):
    text = re.sub(r'\s+', ' ', text.strip())
    sentences = re.split(r'(?<=[.!?؟]) +', text)
    chunks = []
    current_chunk = ""

    for sentence in sentences:
        if len(current_chunk) + len(sentence) <= max_chars:
            current_chunk += sentence + " "
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentence + " "
    if current_chunk:
        chunks.append(current_chunk.strip())
    return chunks


def translate_long_text(text, chunk_size=300, batch_size=6):  # Adjusted batch size and chunk size
    # Split the text into chunks
    chunks = split_text_by_sentences(text, max_chars=chunk_size)
    translated_chunks = []

    print(f"\nTotal chunks to translate: {len(chunks)}")

    # chunks in batches
    for i in range(0, len(chunks), batch_size):
        # Get the current batch of chunks
        batch_chunks = chunks[i:i+batch_size]

        # Tokenize the batch
        inputs = tokenizer(batch_chunks, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)

        try:
            # Generate translation for the batch
            output_ids = model.generate(
                input_ids=inputs['input_ids'],
                attention_mask=inputs['attention_mask'],
                max_length=1024,
                num_beams=5,  # Consider experimenting with num_beams
                no_repeat_ngram_size=3,
                early_stopping=True,
                do_sample=False
            )

            # Decode the translated output and store
            for output in output_ids:
                translated_text = tokenizer.decode(output, skip_special_tokens=True)
                translated_chunks.append(translated_text)

            print(f"Translated batch {i // batch_size + 1} ({len(batch_chunks)} chunks)")

        except Exception as e:
            print(f"Error in batch {i // batch_size + 1}: {e}")
            continue

    # Join
    final_translation = " ".join(translated_chunks)
    final_translation = re.sub(r'\s+', ' ', final_translation).strip()

    return final_translation

In [ ]:
arabic_text = input("Please enter the Arabic text to translate: ")

# translation
translated_text = translate_long_text(arabic_text, chunk_size=300, batch_size=6)
print("\nTranslated Text:\n", translated_text)

Please enter the Arabic text to translate: شهد الطب تطورًا مذهلاً في العقود القليلة الماضية بفضل التقدم التكنولوجي والاكتشافات العلمية. أدت هذه التطورات إلى تحسين طرق التشخيص والعلاج، وزيادة معدلات الشفاء من العديد من الأمراض التي كانت تعتبر مستعصية في الماضي. أصبح بإمكان الأطباء الآن استخدام تقنيات مثل الذكاء الاصطناعي والروبوتات في العمليات الجراحية، مما زاد من دقة الإجراءات الطبية وسرعة تعافي المرضى. كما ساعدت تطبيقات الصحة الرقمية في تمكين الأفراد من مراقبة حالتهم الصحية بشكل مستمر. ومع ذلك، لا تزال هناك تحديات تتعلق بتكلفة الرعاية الصحية وإمكانية الوصول إليها للجميع، مما يتطلب حلولًا مبتكرة وسياسات عادلة لضمان استفادة الجميع من هذه الإنجازات.

Total chunks to translate: 3
Translated batch 1 (3 chunks)

Translated Text:
 medicine has changed quite a lot in the past few years, thanks to technological advance and science. These changes have improved ways of dialytic and cured, and increasing the levels of recovery from many of the ills that were seen as hard in the future. The doctor